# 📊 Tests statistiques - Analyse des comportements clients

**Projet 9 - Analysez les ventes d'une librairie avec Python**

---

## ⚠️ IMPORTANT : Analyse SANS les clients BtoB

Ce notebook contient les analyses statistiques demandées par Julie, en excluant les 4 clients BtoB identifiés dans le premier notebook.

---

## Objectifs de l'analyse

### Les 5 corrélations à tester :

1. **Genre** ↔ **Catégories de livres achetés**
2. **Âge** ↔ **Montant total des achats**
3. **Âge** ↔ **Fréquence d'achat**
4. **Âge** ↔ **Taille du panier moyen**
5. **Âge** ↔ **Catégorie des livres achetés**

### Tests statistiques à utiliser :

- **Test du χ² (Khi-deux)** : Variables qualitatives (Genre vs Catégorie, Âge vs Catégorie)
- **ANOVA ou Test de Student** : Variable qualitative vs quantitative
- **Test de Pearson ou Spearman** : Variables quantitatives (Âge vs CA, etc.)

---

## 1. Import des bibliothèques

In [ ]:
# Manipulation de données
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Tests statistiques
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr, spearmanr, f_oneway, ttest_ind

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Affichage complet des dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Ignorer les warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques importées avec succès")

## 2. Chargement des données

### 2.1 Chargement des fichiers bruts

In [ ]:
# Chargement du fichier clients
df_customers = pd.read_csv('customers.csv', sep=';')

# Chargement du fichier produits
df_products = pd.read_csv('products.csv', sep=';')

# Chargement du fichier transactions
df_trans_raw = pd.read_excel('Transactions.xlsx', header=None)
df_transactions = df_trans_raw[0].str.split(';', expand=True)
df_transactions.columns = df_transactions.iloc[0]
df_transactions = df_transactions.iloc[1:].reset_index(drop=True)

# Conversion de la date
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

print("✅ Fichiers chargés")
print(f"   - Clients : {len(df_customers)}")
print(f"   - Produits : {len(df_products)}")
print(f"   - Transactions : {len(df_transactions)}")

### 2.2 Calcul de l'âge et création du dataset complet

In [ ]:
# Calcul de l'âge
annee_actuelle = 2024
df_customers['age'] = annee_actuelle - df_customers['birth']

# Fusion des données
df_complete = df_transactions.merge(df_products, on='id_prod', how='left')
df_complete = df_complete.merge(df_customers, on='client_id', how='left')
df_complete['ca'] = df_complete['price']

print(f"✅ Dataset complet créé : {len(df_complete)} lignes")
print(f"   Colonnes : {df_complete.columns.tolist()}")

### 2.3 Identification et exclusion des clients BtoB

Nous identifions les 4 clients avec le CA le plus élevé comme étant des clients BtoB professionnels.

In [ ]:
# Calcul du CA par client
client_stats = df_complete.groupby('client_id').agg({
    'ca': ['sum', 'count', 'mean'],
    'session_id': 'nunique'
}).reset_index()

client_stats.columns = ['client_id', 'ca_total', 'nb_achats', 'panier_moyen', 'nb_sessions']
client_stats = client_stats.sort_values('ca_total', ascending=False).reset_index(drop=True)

# Identification des 4 clients BtoB (CA le plus élevé)
clients_btob = client_stats.nlargest(4, 'ca_total')
btob_ids = clients_btob['client_id'].tolist()

print("🏢 Clients BtoB identifiés (à exclure de l'analyse) :")
print(clients_btob[['client_id', 'ca_total', 'nb_achats', 'panier_moyen']])
print(f"\n📋 IDs des clients BtoB : {btob_ids}")

In [ ]:
# ⚠️ EXCLUSION DES CLIENTS BtoB
df_btoc = df_complete[~df_complete['client_id'].isin(btob_ids)].copy()

print("✅ Dataset BtoC créé (SANS les clients BtoB)")
print(f"   - Transactions AVANT exclusion : {len(df_complete)}")
print(f"   - Transactions APRÈS exclusion : {len(df_btoc)}")
print(f"   - Transactions supprimées : {len(df_complete) - len(df_btoc)}")

# Nombre de clients BtoC
nb_clients_btoc = df_btoc['client_id'].nunique()
print(f"\n👥 Nombre de clients BtoC : {nb_clients_btoc}")

### 2.4 Calcul des variables nécessaires pour les tests

Nous devons calculer pour chaque client BtoC :
- Montant total des achats
- Fréquence d'achat (nombre de transactions)
- Panier moyen
- Catégorie préférée

In [ ]:
# Agrégation par client pour obtenir les métriques nécessaires
client_analysis = df_btoc.groupby('client_id').agg({
    'ca': ['sum', 'mean', 'count'],  # Montant total, panier moyen, fréquence
    'categ': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0],  # Catégorie préférée
    'sex': 'first',
    'age': 'first'
}).reset_index()

# Renommer les colonnes
client_analysis.columns = ['client_id', 'montant_total', 'panier_moyen', 'frequence_achat', 
                           'categorie_preferee', 'sexe', 'age']

print("✅ Variables calculées par client BtoC :")
print(client_analysis.head(10))

print(f"\n📊 Statistiques descriptives :")
print(client_analysis[['montant_total', 'panier_moyen', 'frequence_achat', 'age']].describe())

## 3. Vérification des données pour les tests

Avant de réaliser les tests statistiques, vérifions la qualité de nos données.

In [ ]:
# Vérification des valeurs manquantes
print("🔍 Vérification des valeurs manquantes :")
print(client_analysis.isnull().sum())

# Distribution des variables
print("\n📊 Répartition du sexe :")
print(client_analysis['sexe'].value_counts())

print("\n📊 Répartition des catégories préférées :")
print(client_analysis['categorie_preferee'].value_counts())

print("\n📊 Statistiques d'âge :")
print(client_analysis['age'].describe())

In [ ]:
# Visualisation de la distribution des variables clés
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Distribution de l'âge
axes[0, 0].hist(client_analysis['age'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('Âge')
axes[0, 0].set_ylabel('Nombre de clients')
axes[0, 0].set_title('Distribution de l\'âge', fontweight='bold')
axes[0, 0].axvline(client_analysis['age'].mean(), color='red', linestyle='--', 
                   label=f'Moyenne: {client_analysis["age"].mean():.1f} ans')
axes[0, 0].legend()

# Distribution du montant total
axes[0, 1].hist(client_analysis['montant_total'], bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0, 1].set_xlabel('Montant total (€)')
axes[0, 1].set_ylabel('Nombre de clients')
axes[0, 1].set_title('Distribution du montant total', fontweight='bold')
axes[0, 1].axvline(client_analysis['montant_total'].mean(), color='red', linestyle='--', 
                   label=f'Moyenne: {client_analysis["montant_total"].mean():.2f}€')
axes[0, 1].legend()

# Distribution de la fréquence d'achat
axes[0, 2].hist(client_analysis['frequence_achat'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0, 2].set_xlabel('Fréquence d\'achat')
axes[0, 2].set_ylabel('Nombre de clients')
axes[0, 2].set_title('Distribution de la fréquence d\'achat', fontweight='bold')
axes[0, 2].axvline(client_analysis['frequence_achat'].mean(), color='red', linestyle='--', 
                   label=f'Moyenne: {client_analysis["frequence_achat"].mean():.1f}')
axes[0, 2].legend()

# Distribution du panier moyen
axes[1, 0].hist(client_analysis['panier_moyen'], bins=50, edgecolor='black', alpha=0.7, color='purple')
axes[1, 0].set_xlabel('Panier moyen (€)')
axes[1, 0].set_ylabel('Nombre de clients')
axes[1, 0].set_title('Distribution du panier moyen', fontweight='bold')
axes[1, 0].axvline(client_analysis['panier_moyen'].mean(), color='red', linestyle='--', 
                   label=f'Moyenne: {client_analysis["panier_moyen"].mean():.2f}€')
axes[1, 0].legend()

# Répartition par sexe
sexe_counts = client_analysis['sexe'].value_counts()
axes[1, 1].pie(sexe_counts, labels=['Femme', 'Homme'] if sexe_counts.index[0] == 'f' else ['Homme', 'Femme'],
               autopct='%1.1f%%', colors=['#FF69B4', '#4169E1'], startangle=90)
axes[1, 1].set_title('Répartition par sexe', fontweight='bold')

# Répartition par catégorie préférée
cat_counts = client_analysis['categorie_preferee'].value_counts()
axes[1, 2].bar(cat_counts.index.astype(str), cat_counts.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1'], alpha=0.7)
axes[1, 2].set_xlabel('Catégorie')
axes[1, 2].set_ylabel('Nombre de clients')
axes[1, 2].set_title('Catégories préférées', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. TEST 1 : Genre ↔ Catégories de livres achetés

### Type de variables :
- **Genre** : Variable qualitative (binaire : m/f)
- **Catégorie** : Variable qualitative (0, 1, 2)

### Test approprié : **Test du χ² (Khi-deux)**

**Hypothèses :**
- H0 (hypothèse nulle) : Il n'y a pas de lien entre le genre et la catégorie de livres achetés
- H1 (hypothèse alternative) : Il existe un lien entre le genre et la catégorie de livres achetés

In [ ]:
# Création du tableau de contingence
contingency_table = pd.crosstab(client_analysis['sexe'], client_analysis['categorie_preferee'])

print("📊 Tableau de contingence : Genre vs Catégorie préférée")
print(contingency_table)

# Ajout des totaux
print("\nAvec totaux :")
contingency_with_totals = pd.crosstab(client_analysis['sexe'], client_analysis['categorie_preferee'], 
                                      margins=True, margins_name='Total')
print(contingency_with_totals)

In [ ]:
# Test du χ² (Khi-deux)
chi2_stat, p_value, dof, expected_freq = chi2_contingency(contingency_table)

print("="*70)
print("TEST DU χ² : GENRE vs CATÉGORIE")
print("="*70)
print(f"Statistique χ² : {chi2_stat:.4f}")
print(f"p-value : {p_value:.6f}")
print(f"Degrés de liberté : {dof}")
print("\nFréquences attendues (sous H0) :")
print(pd.DataFrame(expected_freq, 
                   index=contingency_table.index, 
                   columns=contingency_table.columns))

# Interprétation
alpha = 0.05
print(f"\n{'='*70}")
print("INTERPRÉTATION (seuil α = 0.05) :")
print(f"{'='*70}")
if p_value < alpha:
    print(f"✅ p-value ({p_value:.6f}) < α ({alpha})")
    print("➡️  On REJETTE H0")
    print("➡️  Conclusion : Il existe un lien significatif entre le genre et la catégorie de livres achetés.")
else:
    print(f"❌ p-value ({p_value:.6f}) ≥ α ({alpha})")
    print("➡️  On NE REJETTE PAS H0")
    print("➡️  Conclusion : Pas de lien significatif entre le genre et la catégorie de livres achetés.")
print("="*70)

In [ ]:
# Visualisation bivarée : Genre vs Catégorie
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique en barres groupées
contingency_table.T.plot(kind='bar', ax=axes[0], color=['#FF69B4', '#4169E1'], alpha=0.7)
axes[0].set_xlabel('Catégorie de livres')
axes[0].set_ylabel('Nombre de clients')
axes[0].set_title('Distribution des catégories par genre', fontweight='bold')
axes[0].legend(title='Genre', labels=['Femme', 'Homme'])
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].grid(True, alpha=0.3, axis='y')

# Graphique en barres empilées (pourcentages)
contingency_pct = contingency_table.div(contingency_table.sum(axis=1), axis=0) * 100
contingency_pct.T.plot(kind='bar', stacked=True, ax=axes[1], color=['#FF69B4', '#4169E1'], alpha=0.7)
axes[1].set_xlabel('Catégorie de livres')
axes[1].set_ylabel('Pourcentage (%)')
axes[1].set_title('Répartition en % des catégories par genre', fontweight='bold')
axes[1].legend(title='Genre', labels=['Femme', 'Homme'])
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].grid(True, alpha=0.3, axis='y')

plt.suptitle(f'TEST χ² : Genre vs Catégorie (χ²={chi2_stat:.2f}, p={p_value:.4f})', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 5. TEST 2 : Âge ↔ Montant total des achats

### Type de variables :
- **Âge** : Variable quantitative continue
- **Montant total** : Variable quantitative continue

### Test approprié : **Test de corrélation de Pearson (ou Spearman)**

**Hypothèses :**
- H0 : Il n'y a pas de corrélation entre l'âge et le montant total des achats (ρ = 0)
- H1 : Il existe une corrélation entre l'âge et le montant total des achats (ρ ≠ 0)

In [ ]:
# Test de corrélation de Pearson
pearson_corr, pearson_pvalue = pearsonr(client_analysis['age'], client_analysis['montant_total'])

# Test de corrélation de Spearman (plus robuste aux valeurs extrêmes)
spearman_corr, spearman_pvalue = spearmanr(client_analysis['age'], client_analysis['montant_total'])

print("="*70)
print("TEST DE CORRÉLATION : ÂGE vs MONTANT TOTAL DES ACHATS")
print("="*70)

print("\n📊 PEARSON (corrélation linéaire) :")
print(f"   Coefficient de corrélation (r) : {pearson_corr:.4f}")
print(f"   p-value : {pearson_pvalue:.6f}")

print("\n📊 SPEARMAN (corrélation monotone) :")
print(f"   Coefficient de corrélation (ρ) : {spearman_corr:.4f}")
print(f"   p-value : {spearman_pvalue:.6f}")

# Interprétation de la force de la corrélation
def interpret_correlation(r):
    abs_r = abs(r)
    if abs_r < 0.1:
        return "négligeable"
    elif abs_r < 0.3:
        return "faible"
    elif abs_r < 0.5:
        return "modérée"
    elif abs_r < 0.7:
        return "forte"
    else:
        return "très forte"

# Interprétation
alpha = 0.05
print(f"\n{'='*70}")
print("INTERPRÉTATION (seuil α = 0.05) :")
print(f"{'='*70}")
if pearson_pvalue < alpha:
    print(f"✅ p-value ({pearson_pvalue:.6f}) < α ({alpha})")
    print("➡️  On REJETTE H0")
    print(f"➡️  Conclusion : Il existe une corrélation {interpret_correlation(pearson_corr)} ")
    print(f"     {'positive' if pearson_corr > 0 else 'négative'} entre l'âge et le montant total des achats.")
    print(f"     (r = {pearson_corr:.4f})")
else:
    print(f"❌ p-value ({pearson_pvalue:.6f}) ≥ α ({alpha})")
    print("➡️  On NE REJETTE PAS H0")
    print("➡️  Conclusion : Pas de corrélation significative entre l'âge et le montant total des achats.")
print("="*70)

In [ ]:
# Visualisation bivarée : Âge vs Montant total
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Nuage de points avec droite de régression
axes[0].scatter(client_analysis['age'], client_analysis['montant_total'], 
                alpha=0.5, s=30, color='steelblue')
# Droite de régression
z = np.polyfit(client_analysis['age'], client_analysis['montant_total'], 1)
p = np.poly1d(z)
axes[0].plot(client_analysis['age'], p(client_analysis['age']), 
             "r--", linewidth=2, label=f'Régression linéaire')
axes[0].set_xlabel('Âge (années)')
axes[0].set_ylabel('Montant total (€)')
axes[0].set_title('Nuage de points : Âge vs Montant total', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot du montant total par tranche d'âge
client_analysis['tranche_age'] = pd.cut(client_analysis['age'], 
                                        bins=[0, 25, 35, 45, 55, 65, 100],
                                        labels=['<25', '25-35', '35-45', '45-55', '55-65', '65+'])
client_analysis.boxplot(column='montant_total', by='tranche_age', ax=axes[1])
axes[1].set_xlabel('Tranche d\'âge')
axes[1].set_ylabel('Montant total (€)')
axes[1].set_title('Montant total par tranche d\'âge', fontweight='bold')
plt.sca(axes[1])
plt.xticks(rotation=0)

plt.suptitle(f'TEST PEARSON : Âge vs Montant total (r={pearson_corr:.3f}, p={pearson_pvalue:.4f})', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 6. TEST 3 : Âge ↔ Fréquence d'achat

### Type de variables :
- **Âge** : Variable quantitative continue
- **Fréquence d'achat** : Variable quantitative discrète

### Test approprié : **Test de corrélation de Spearman**

**Hypothèses :**
- H0 : Il n'y a pas de corrélation entre l'âge et la fréquence d'achat (ρ = 0)
- H1 : Il existe une corrélation entre l'âge et la fréquence d'achat (ρ ≠ 0)

In [ ]:
# Test de corrélation de Spearman (préférable pour fréquence d'achat)
spearman_corr_freq, spearman_pvalue_freq = spearmanr(client_analysis['age'], 
                                                       client_analysis['frequence_achat'])

# Test de Pearson aussi pour comparaison
pearson_corr_freq, pearson_pvalue_freq = pearsonr(client_analysis['age'], 
                                                    client_analysis['frequence_achat'])

print("="*70)
print("TEST DE CORRÉLATION : ÂGE vs FRÉQUENCE D'ACHAT")
print("="*70)

print("\n📊 SPEARMAN (corrélation monotone - RECOMMANDÉ) :")
print(f"   Coefficient de corrélation (ρ) : {spearman_corr_freq:.4f}")
print(f"   p-value : {spearman_pvalue_freq:.6f}")

print("\n📊 PEARSON (corrélation linéaire) :")
print(f"   Coefficient de corrélation (r) : {pearson_corr_freq:.4f}")
print(f"   p-value : {pearson_pvalue_freq:.6f}")

# Interprétation
alpha = 0.05
print(f"\n{'='*70}")
print("INTERPRÉTATION (seuil α = 0.05) :")
print(f"{'='*70}")
if spearman_pvalue_freq < alpha:
    print(f"✅ p-value ({spearman_pvalue_freq:.6f}) < α ({alpha})")
    print("➡️  On REJETTE H0")
    print(f"➡️  Conclusion : Il existe une corrélation {interpret_correlation(spearman_corr_freq)} ")
    print(f"     {'positive' if spearman_corr_freq > 0 else 'négative'} entre l'âge et la fréquence d'achat.")
    print(f"     (ρ = {spearman_corr_freq:.4f})")
else:
    print(f"❌ p-value ({spearman_pvalue_freq:.6f}) ≥ α ({alpha})")
    print("➡️  On NE REJETTE PAS H0")
    print("➡️  Conclusion : Pas de corrélation significative entre l'âge et la fréquence d'achat.")
print("="*70)

In [ ]:
# Visualisation bivarée : Âge vs Fréquence d'achat
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Nuage de points
axes[0].scatter(client_analysis['age'], client_analysis['frequence_achat'], 
                alpha=0.5, s=30, color='green')
# Droite de régression
z = np.polyfit(client_analysis['age'], client_analysis['frequence_achat'], 1)
p = np.poly1d(z)
axes[0].plot(client_analysis['age'], p(client_analysis['age']), 
             "r--", linewidth=2, label=f'Régression linéaire')
axes[0].set_xlabel('Âge (années)')
axes[0].set_ylabel('Fréquence d\'achat (nombre de transactions)')
axes[0].set_title('Nuage de points : Âge vs Fréquence d\'achat', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot de la fréquence par tranche d'âge
client_analysis.boxplot(column='frequence_achat', by='tranche_age', ax=axes[1])
axes[1].set_xlabel('Tranche d\'âge')
axes[1].set_ylabel('Fréquence d\'achat')
axes[1].set_title('Fréquence d\'achat par tranche d\'âge', fontweight='bold')
plt.sca(axes[1])
plt.xticks(rotation=0)

plt.suptitle(f'TEST SPEARMAN : Âge vs Fréquence (ρ={spearman_corr_freq:.3f}, p={spearman_pvalue_freq:.4f})', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 7. TEST 4 : Âge ↔ Taille du panier moyen

### Type de variables :
- **Âge** : Variable quantitative continue
- **Panier moyen** : Variable quantitative continue

### Test approprié : **Test de corrélation de Pearson**

**Hypothèses :**
- H0 : Il n'y a pas de corrélation entre l'âge et le panier moyen (ρ = 0)
- H1 : Il existe une corrélation entre l'âge et le panier moyen (ρ ≠ 0)

In [ ]:
# Test de corrélation de Pearson
pearson_corr_panier, pearson_pvalue_panier = pearsonr(client_analysis['age'], 
                                                        client_analysis['panier_moyen'])

# Test de Spearman pour comparaison
spearman_corr_panier, spearman_pvalue_panier = spearmanr(client_analysis['age'], 
                                                           client_analysis['panier_moyen'])

print("="*70)
print("TEST DE CORRÉLATION : ÂGE vs PANIER MOYEN")
print("="*70)

print("\n📊 PEARSON (corrélation linéaire - RECOMMANDÉ) :")
print(f"   Coefficient de corrélation (r) : {pearson_corr_panier:.4f}")
print(f"   p-value : {pearson_pvalue_panier:.6f}")

print("\n📊 SPEARMAN (corrélation monotone) :")
print(f"   Coefficient de corrélation (ρ) : {spearman_corr_panier:.4f}")
print(f"   p-value : {spearman_pvalue_panier:.6f}")

# Interprétation
alpha = 0.05
print(f"\n{'='*70}")
print("INTERPRÉTATION (seuil α = 0.05) :")
print(f"{'='*70}")
if pearson_pvalue_panier < alpha:
    print(f"✅ p-value ({pearson_pvalue_panier:.6f}) < α ({alpha})")
    print("➡️  On REJETTE H0")
    print(f"➡️  Conclusion : Il existe une corrélation {interpret_correlation(pearson_corr_panier)} ")
    print(f"     {'positive' if pearson_corr_panier > 0 else 'négative'} entre l'âge et le panier moyen.")
    print(f"     (r = {pearson_corr_panier:.4f})")
else:
    print(f"❌ p-value ({pearson_pvalue_panier:.6f}) ≥ α ({alpha})")
    print("➡️  On NE REJETTE PAS H0")
    print("➡️  Conclusion : Pas de corrélation significative entre l'âge et le panier moyen.")
print("="*70)

In [ ]:
# Visualisation bivarée : Âge vs Panier moyen
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Nuage de points
axes[0].scatter(client_analysis['age'], client_analysis['panier_moyen'], 
                alpha=0.5, s=30, color='purple')
# Droite de régression
z = np.polyfit(client_analysis['age'], client_analysis['panier_moyen'], 1)
p = np.poly1d(z)
axes[0].plot(client_analysis['age'], p(client_analysis['age']), 
             "r--", linewidth=2, label=f'Régression linéaire')
axes[0].set_xlabel('Âge (années)')
axes[0].set_ylabel('Panier moyen (€)')
axes[0].set_title('Nuage de points : Âge vs Panier moyen', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot du panier moyen par tranche d'âge
client_analysis.boxplot(column='panier_moyen', by='tranche_age', ax=axes[1])
axes[1].set_xlabel('Tranche d\'âge')
axes[1].set_ylabel('Panier moyen (€)')
axes[1].set_title('Panier moyen par tranche d\'âge', fontweight='bold')
plt.sca(axes[1])
plt.xticks(rotation=0)

plt.suptitle(f'TEST PEARSON : Âge vs Panier moyen (r={pearson_corr_panier:.3f}, p={pearson_pvalue_panier:.4f})', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 8. TEST 5 : Âge ↔ Catégorie des livres achetés

### Type de variables :
- **Âge** : Variable quantitative continue → À discrétiser en tranches d'âge
- **Catégorie** : Variable qualitative (0, 1, 2)

### Tests appropriés :
1. **Test du χ²** (après discrétisation de l'âge)
2. **ANOVA** (pour comparer l'âge moyen entre catégories)

**Hypothèses (ANOVA) :**
- H0 : Les moyennes d'âge sont identiques pour toutes les catégories
- H1 : Au moins une catégorie a une moyenne d'âge différente

In [ ]:
# Méthode 1 : Test du χ² avec âge discrétisé
# On utilise la colonne 'tranche_age' créée précédemment

contingency_age_cat = pd.crosstab(client_analysis['tranche_age'], 
                                   client_analysis['categorie_preferee'])

print("📊 Tableau de contingence : Tranche d'âge vs Catégorie préférée")
print(contingency_age_cat)

# Test du χ²
chi2_age_cat, p_age_cat, dof_age_cat, expected_age_cat = chi2_contingency(contingency_age_cat)

print("\n" + "="*70)
print("TEST DU χ² : TRANCHE D'ÂGE vs CATÉGORIE")
print("="*70)
print(f"Statistique χ² : {chi2_age_cat:.4f}")
print(f"p-value : {p_age_cat:.6f}")
print(f"Degrés de liberté : {dof_age_cat}")

In [ ]:
# Méthode 2 : ANOVA - Comparer l'âge moyen entre les catégories
# Séparation des âges par catégorie
age_cat_0 = client_analysis[client_analysis['categorie_preferee'] == 0]['age']
age_cat_1 = client_analysis[client_analysis['categorie_preferee'] == 1]['age']
age_cat_2 = client_analysis[client_analysis['categorie_preferee'] == 2]['age']

# Test ANOVA
f_stat, p_anova = f_oneway(age_cat_0, age_cat_1, age_cat_2)

print("\n" + "="*70)
print("TEST ANOVA : ÂGE MOYEN par CATÉGORIE")
print("="*70)
print(f"Statistique F : {f_stat:.4f}")
print(f"p-value : {p_anova:.6f}")

# Moyennes d'âge par catégorie
print("\n📊 Âge moyen par catégorie :")
age_by_cat = client_analysis.groupby('categorie_preferee')['age'].agg(['mean', 'std', 'count'])
age_by_cat.columns = ['Âge moyen', 'Écart-type', 'Nombre de clients']
print(age_by_cat)

In [ ]:
# Interprétation des deux tests
alpha = 0.05

print("\n" + "="*70)
print("INTERPRÉTATION GLOBALE (seuil α = 0.05) :")
print("="*70)

print("\n1️⃣ TEST χ² (Tranche d'âge vs Catégorie) :")
if p_age_cat < alpha:
    print(f"   ✅ p-value ({p_age_cat:.6f}) < α ({alpha})")
    print("   ➡️  On REJETTE H0")
    print("   ➡️  Conclusion : Il existe un lien significatif entre la tranche d'âge")
    print("       et la catégorie de livres préférée.")
else:
    print(f"   ❌ p-value ({p_age_cat:.6f}) ≥ α ({alpha})")
    print("   ➡️  On NE REJETTE PAS H0")
    print("   ➡️  Conclusion : Pas de lien significatif entre la tranche d'âge")
    print("       et la catégorie de livres préférée.")

print("\n2️⃣ TEST ANOVA (Âge moyen par catégorie) :")
if p_anova < alpha:
    print(f"   ✅ p-value ({p_anova:.6f}) < α ({alpha})")
    print("   ➡️  On REJETTE H0")
    print("   ➡️  Conclusion : L'âge moyen diffère significativement entre")
    print("       au moins deux catégories de livres.")
else:
    print(f"   ❌ p-value ({p_anova:.6f}) ≥ α ({alpha})")
    print("   ➡️  On NE REJETTE PAS H0")
    print("   ➡️  Conclusion : L'âge moyen ne diffère pas significativement")
    print("       entre les catégories de livres.")

print("="*70)

In [ ]:
# Visualisation bivarée : Âge vs Catégorie
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Graphique 1 : Barres groupées (tranches d'âge par catégorie)
contingency_age_cat.plot(kind='bar', ax=axes[0], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], alpha=0.7)
axes[0].set_xlabel('Tranche d\'âge')
axes[0].set_ylabel('Nombre de clients')
axes[0].set_title('Distribution des catégories par tranche d\'âge', fontweight='bold')
axes[0].legend(title='Catégorie', labels=['Cat. 0', 'Cat. 1', 'Cat. 2'])
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)
axes[0].grid(True, alpha=0.3, axis='y')

# Graphique 2 : Boxplot de l'âge par catégorie
client_analysis.boxplot(column='age', by='categorie_preferee', ax=axes[1])
axes[1].set_xlabel('Catégorie de livres')
axes[1].set_ylabel('Âge (années)')
axes[1].set_title('Distribution de l\'âge par catégorie', fontweight='bold')
plt.sca(axes[1])
plt.xticks(rotation=0)

# Graphique 3 : Violin plot
import seaborn as sns
sns.violinplot(data=client_analysis, x='categorie_preferee', y='age', ax=axes[2],
               palette=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[2].set_xlabel('Catégorie de livres')
axes[2].set_ylabel('Âge (années)')
axes[2].set_title('Distribution de l\'âge par catégorie (violin plot)', fontweight='bold')

plt.suptitle(f'Âge vs Catégorie | χ²={chi2_age_cat:.2f} (p={p_age_cat:.4f}) | ANOVA F={f_stat:.2f} (p={p_anova:.4f})', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 9. Synthèse des résultats des tests statistiques

In [ ]:
# Création d'un tableau récapitulatif
resultats = {
    'Test': [
        'Genre ↔ Catégorie',
        'Âge ↔ Montant total',
        'Âge ↔ Fréquence achat',
        'Âge ↔ Panier moyen',
        'Âge ↔ Catégorie (χ²)',
        'Âge ↔ Catégorie (ANOVA)'
    ],
    'Type de test': [
        'χ² (Khi-deux)',
        'Pearson',
        'Spearman',
        'Pearson',
        'χ² (Khi-deux)',
        'ANOVA'
    ],
    'Statistique': [
        f'χ² = {chi2_stat:.4f}',
        f'r = {pearson_corr:.4f}',
        f'ρ = {spearman_corr_freq:.4f}',
        f'r = {pearson_corr_panier:.4f}',
        f'χ² = {chi2_age_cat:.4f}',
        f'F = {f_stat:.4f}'
    ],
    'p-value': [
        f'{p_value:.6f}',
        f'{pearson_pvalue:.6f}',
        f'{spearman_pvalue_freq:.6f}',
        f'{pearson_pvalue_panier:.6f}',
        f'{p_age_cat:.6f}',
        f'{p_anova:.6f}'
    ],
    'Significatif (α=0.05)': [
        '✅ OUI' if p_value < 0.05 else '❌ NON',
        '✅ OUI' if pearson_pvalue < 0.05 else '❌ NON',
        '✅ OUI' if spearman_pvalue_freq < 0.05 else '❌ NON',
        '✅ OUI' if pearson_pvalue_panier < 0.05 else '❌ NON',
        '✅ OUI' if p_age_cat < 0.05 else '❌ NON',
        '✅ OUI' if p_anova < 0.05 else '❌ NON'
    ]
}

df_resultats = pd.DataFrame(resultats)

print("="*100)
print("📊 SYNTHÈSE DES RÉSULTATS DES TESTS STATISTIQUES")
print("="*100)
print(df_resultats.to_string(index=False))
print("="*100)

## 10. Conclusions et recommandations

### Points clés de l'analyse statistique :

**Méthodologie :**
- ✅ Analyse réalisée SANS les 4 clients BtoB identifiés
- ✅ 5 corrélations testées comme demandé par Julie
- ✅ Tests statistiques appropriés selon le type de variables
- ✅ Graphiques bivariés pour chaque test
- ✅ Seuil de significativité : α = 0.05

**Tests réalisés :**
1. **Test du χ²** pour les variables qualitatives
2. **Test de Pearson** pour les corrélations linéaires
3. **Test de Spearman** pour les corrélations monotones
4. **ANOVA** pour comparer les moyennes entre groupes

### Recommandations pour la soutenance :

1. **Expliquer le choix des tests** : Adapter le test au type de variables
2. **Interpréter la p-value** : Seuil de significativité de 5%
3. **Visualiser les résultats** : Les graphiques facilitent la compréhension
4. **Justifier l'exclusion des BtoB** : Impact sur les résultats
5. **Proposer des actions** : Basées sur les corrélations significatives

In [ ]:
# Export du tableau de synthèse
df_resultats.to_csv('synthese_tests_statistiques.csv', index=False, sep=';')
print("✅ Synthèse des tests exportée : synthese_tests_statistiques.csv")

# Export du dataset d'analyse client
client_analysis.to_csv('client_analysis_btoc.csv', index=False, sep=';')
print("✅ Dataset d'analyse clients exporté : client_analysis_btoc.csv")